In [16]:
import os
import sys
from subprocess import check_output

def decrypt_byte(ciphertext, y_n, y_n_minus_1):
    for i in range(256):
        r = os.urandom(15) + bytes([i])
        modified_ciphertext = y_n_minus_1 + r + y_n
        with open("temp_cipher.bin", "wb") as f:
            f.write(modified_ciphertext)
        result = check_output(['python3', 'oracle.py', 'temp_cipher.bin']).strip()
        print(f"Testing byte value: {i}, Result: {result}")  # Debug print
        if result == b'1':
            return i
    raise Exception("Failed to decrypt the byte")

def decrypt_block(ciphertext, y_n, y_n_minus_1):
    decrypted_bytes = bytearray(16)
    derived_values = bytearray(16)
    derived_values[15] = decrypt_byte(ciphertext, y_n, y_n_minus_1)
    decrypted_bytes[15] = derived_values[15] ^ y_n_minus_1[15]

    for k in range(14, -1, -1):
        padding_value = 17 - (k + 1)
        r = (os.urandom(k) + bytes([derived_values[k+1] ^ padding_value]) +
             bytes([derived_values[j] ^ padding_value for j in range(k+1, 16)]))
        modified_ciphertext = y_n_minus_1 + r + y_n
        with open("temp_cipher.bin", "wb") as f:
            f.write(modified_ciphertext)
        result = check_output(['python3', 'oracle.py', 'temp_cipher.bin']).strip()
        print(f"Decrypting byte {k}, Result: {result}")  # Debug print
        if result == b'1':
            derived_values[k] = r[k] ^ padding_value
            decrypted_bytes[k] = derived_values[k] ^ y_n_minus_1[k]
    return decrypted_bytes

def decrypt(ciphertext):
    blocks = [ciphertext[i:i+16] for i in range(0, len(ciphertext), 16)]
    iv = blocks[0]  # The IV should not be decrypted but used to decrypt the first block
    plaintext = b''

    # Start decryption from the second block
    for i in range(1, len(blocks)):
        plaintext += decrypt_block(ciphertext, blocks[i], blocks[i-1])

    # Assume the last byte indicates padding length and check for valid padding
    pad_len = plaintext[-1]
    if pad_len < 1 or pad_len > 16:
        raise ValueError("Invalid padding length.")
    if all(plaintext[-j] == pad_len for j in range(1, pad_len+1)):
        return plaintext[:-pad_len]
    else:
        raise ValueError("Padding does not match expected format.")
    
def decrypt_test_block(ciphertext, y_n, y_n_minus_1):
    for i in range(256):
        r = os.urandom(15) + bytes([i])
        modified_ciphertext = y_n_minus_1 + r + y_n
        result = check_output(['python3', 'oracle.py', modified_ciphertext]).strip()
        if result == '1':
            return i, modified_ciphertext
    return None, None



In [17]:
with open("ciphertext", "rb") as file:
        ciphertext = file.read()
    
if len(ciphertext) % 16 != 0 or len(ciphertext) < 32:
    print("Ciphertext must be a multiple of 16 bytes and at least 32 bytes.")
    sys.exit(1)
    
try:
    plaintext = decrypt(ciphertext)
    plaintext_decoded = plaintext.decode('ascii')  # Decode bytes to string for ASCII plaintext

    # Now, write the decrypted plaintext to a file
    with open("plaintext.txt", "w") as text_file:
        text_file.write(plaintext_decoded)
    print("Decrypted plaintext saved to plaintext.txt")
except ValueError as e:
    print(str(e))

Testing byte value: 0, Result: b'0'
Testing byte value: 1, Result: b'0'
Testing byte value: 2, Result: b'0'
Testing byte value: 3, Result: b'0'
Testing byte value: 4, Result: b'0'
Testing byte value: 5, Result: b'0'
Testing byte value: 6, Result: b'0'
Testing byte value: 7, Result: b'0'
Testing byte value: 8, Result: b'0'
Testing byte value: 9, Result: b'0'
Testing byte value: 10, Result: b'0'
Testing byte value: 11, Result: b'0'
Testing byte value: 12, Result: b'0'
Testing byte value: 13, Result: b'0'
Testing byte value: 14, Result: b'0'
Testing byte value: 15, Result: b'0'
Testing byte value: 16, Result: b'0'
Testing byte value: 17, Result: b'0'
Testing byte value: 18, Result: b'0'
Testing byte value: 19, Result: b'0'
Testing byte value: 20, Result: b'0'
Testing byte value: 21, Result: b'0'
Testing byte value: 22, Result: b'0'
Testing byte value: 23, Result: b'0'
Testing byte value: 24, Result: b'0'
Testing byte value: 25, Result: b'0'
Testing byte value: 26, Result: b'0'
Testing byt

In [18]:
# Simplified decryption function to test smaller segments and ensure correct block passing
iv_bytes = b'CMPT 403 Test IV'

# Testing a smaller part to debug and optimize
test_byte, test_ciphertext = decrypt_test_block(ciphertext[16:32], ciphertext[16:32], b'CMPT 403 Test IV')
test_byte, test_ciphertext

# Manually construct a block with correct and incorrect padding
correct_padding_block = ciphertext[16:32]  # assuming this block ends with correct padding
incorrect_padding_block = correct_padding_block[:-1] + bytes([correct_padding_block[-1] ^ 1])  # Flip last byte

# Test the oracle with both
oracle_result_correct = check_output(['python3', 'oracle.py', iv_bytes + correct_padding_block]).strip()
oracle_result_incorrect = check_output(['python3', 'oracle.py', iv_bytes + correct_padding_block]).strip()

oracle_result_correct, oracle_result_incorrect

ValueError: embedded null byte